In [11]:
import numpy as np
import pickle

# Ensure the plotting backend is set correctly
%matplotlib inline

In [3]:
backup_file = 'data/grouped_dfs.pkl'
try:
    with open(backup_file, 'rb') as f:
        grouped_dfs = pickle.load(f)
except FileNotFoundError:
    print('Backup file not found. Rebuilding the data.')

# Spot MB

### Continuation Setup
- Focus on the 2nd or 3rd breakout
- Stock close near the high
- Day before the breakout, negative or small up day
- 3 to 10 days
- Not Up 2 days in the row
- Highest probability of success
- 2lynch

In [17]:
df_wwww = grouped_dfs['WWW']['2022-12-03':'2023-02-21']
df_wwww.to_csv('data/wwww.csv')

In [18]:
df_amgn = grouped_dfs['AMGN']['2022-10-17':'2023-11-14']
df_tree = grouped_dfs['TREE']['2022-12-12':'2023-02-27']
df_fslr = grouped_dfs['FSLR']['2022-10-03':'2023-01-01']


#### 2 Lynch

In [19]:
mb_tests = {
    'AMGN': df_amgn,
    'TREE': df_tree,
    'FSLR': df_fslr,
    'WWW': df_wwww
}

In [20]:
# Input values
candle_body_threshold = 0.6
open_to_high_threshold = 2.0
volume_sma_length = 20
high_lookback_period_10 = 10
high_lookback_period_5 = 5
low_lookback_period_20 = 20
low_threshold_percentage = 10.0
low_above_lookback_low_percentage = 5.0
move_threshold = 4.0

In [21]:
def is_narrow(df_):
    # rename columns to lowercase
    df_.columns = df_.columns.str.lower()
    # Calculate candle body percentage and open to high percentage
    df_['candle_body_percentage'] = np.abs(df_['close'] - df_['open']) / df_['open'] * 100
    df_['open_to_high_percentage'] = (df_['high'] - df_['open']) / df_['open'] * 100
    
    # Calculate average volume
    df_['avg_volume'] = df_['volume'].rolling(window=volume_sma_length).mean()
    
    # Calculate the highest high over the last 10 days
    df_['highest_high_last_10_days'] = df_['high'].rolling(window=high_lookback_period_10).max()
    
    # Calculate the highest high over the last 5 days
    df_['highest_high_last_5_days'] = df_['high'].rolling(window=high_lookback_period_5).max()
    
    # Calculate the lowest low over the last 20 days
    df_['lowest_low_last_20_days'] = df_['low'].rolling(window=low_lookback_period_20).min()
    
    # Calculate the low threshold value
    df_['low_threshold_value'] = df_['highest_high_last_10_days'] * (1 - low_threshold_percentage / 100)
    
    # Calculate the threshold where the low must be above the lowest low of the last 20 days
    df_['low_above_lookback_low_value'] = df_['lowest_low_last_20_days'] * (1 + low_above_lookback_low_percentage / 100)
    
    # Condition for identifying narrow candles
    df_['is_narrow_candle'] = (
        (df_['candle_body_percentage'] < candle_body_threshold) & 
        (df_['open_to_high_percentage'] < open_to_high_threshold) & 
        (df_['volume'] < df_['avg_volume']) &
        (df_['low'] >= df_['low_threshold_value']) &
        (df_['high'] < df_['highest_high_last_5_days']) &
        (df_['low'] > df_['low_above_lookback_low_value'])
    )
    return df_

In [22]:
df_amgn

Price,Close,High,Low,Open,Volume
Date,,,,,
2022-10-17,252.929993,253.419998,249.490005,251.419998,2577100.0
2022-10-18,252.119995,254.699997,251.029999,253.860001,2060000.0
2022-10-19,248.190002,252.350006,246.940002,251.309998,1883000.0
2022-10-20,247.449997,249.059998,246.009995,247.550003,2058000.0
2022-10-21,251.940002,252.389999,247.009995,247.520004,3531200.0
...,...,...,...,...,...
2023-11-08,273.260010,274.769989,270.750000,272.420013,1916800.0
2023-11-09,264.059998,273.230011,263.700012,272.130005,2266000.0
2023-11-10,267.309998,267.940002,262.510010,266.640015,2070600.0


In [23]:
df_amgn = is_narrow(df_amgn)
df_amgn

C:\Users\otrem\AppData\Local\Temp\ipykernel_6324\3751688661.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['candle_body_percentage'] = np.abs(df_['close'] - df_['open']) / df_['open'] * 100
C:\Users\otrem\AppData\Local\Temp\ipykernel_6324\3751688661.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['open_to_high_percentage'] = (df_['high'] - df_['open']) / df_['open'] * 100
C:\Users\otrem\AppData\Local\Temp\ipykernel_6324\3751688661.py:9: SettingWithCopyWarning: 
A value is trying to be set o

Price,close,high,low,open,volume,candle_body_percentage,open_to_high_percentage,avg_volume,highest_high_last_10_days,highest_high_last_5_days,lowest_low_last_20_days,low_threshold_value,low_above_lookback_low_value,is_narrow_candle
Date,,,,,,,,,,,,,,
2022-10-17,252.929993,253.419998,249.490005,251.419998,2577100.0,0.600586,0.795482,NaN,NaN,NaN,NaN,NaN,NaN,False
2022-10-18,252.119995,254.699997,251.029999,253.860001,2060000.0,0.685419,0.330890,NaN,NaN,NaN,NaN,NaN,NaN,False
2022-10-19,248.190002,252.350006,246.940002,251.309998,1883000.0,1.241493,0.413835,NaN,NaN,NaN,NaN,NaN,NaN,False
2022-10-20,247.449997,249.059998,246.009995,247.550003,2058000.0,0.040398,0.609976,NaN,NaN,NaN,NaN,NaN,NaN,False
2022-10-21,251.940002,252.389999,247.009995,247.520004,3531200.0,1.785714,1.967516,NaN,NaN,254.699997,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-08,273.260010,274.769989,270.750000,272.420013,1916800.0,0.308346,0.862630,2716170.0,274.950012,274.950012,249.699997,247.455011,262.184997,True
2023-11-09,264.059998,273.230011,263.700012,272.130005,2266000.0,2.965497,0.404221,2676840.0,274.950012,274.950012,249.699997,247.455011,262.184997,False
2023-11-10,267.309998,267.940002,262.510010,266.640015,2070600.0,0.251269,0.487544,2606710.0,274.950012,274.950012,249.699997,247.455011,262.184997,True


In [24]:
# rows where is narrow candle is True
df_amgn[df_amgn['is_narrow_candle']]

Price,close,high,low,open,volume,candle_body_percentage,open_to_high_percentage,avg_volume,highest_high_last_10_days,highest_high_last_5_days,lowest_low_last_20_days,low_threshold_value,low_above_lookback_low_value,is_narrow_candle
Date,,,,,,,,,,,,,,
2022-11-16,283.769989,286.989990,282.559998,284.019989,2443500.0,0.088022,1.045701,3089755.0,296.670013,294.480011,246.009995,267.003012,258.310494,True
2022-11-29,281.989990,283.380005,279.679993,283.380005,2135300.0,0.490513,0.000000,2956430.0,291.600006,289.540009,259.839996,262.440005,272.831996,True
2022-12-02,285.510010,286.119995,283.190002,284.059998,2267000.0,0.510460,0.725198,3130350.0,291.600006,288.279999,259.839996,262.440005,272.831996,True
2023-04-11,251.429993,253.080002,250.979996,251.419998,1272800.0,0.003975,0.660251,2598300.0,256.440002,256.440002,228.520004,230.796002,239.946004,True
2023-04-14,250.000000,251.759995,248.960007,250.619995,1866200.0,0.247385,0.454872,2473830.0,256.440002,256.440002,228.520004,230.796002,239.946004,True
2023-04-19,246.210007,246.759995,244.369995,245.250000,1246500.0,0.391440,0.615696,2109015.0,256.440002,251.899994,229.589996,230.796002,241.069496,True
2023-06-20,228.589996,230.839996,227.190002,229.029999,2492700.0,0.192116,0.790288,2861570.0,231.949997,231.949997,211.710007,208.754997,222.295507,True
2023-06-21,227.039993,230.000000,225.589996,228.179993,2282600.0,0.499605,0.797619,2883270.0,231.949997,231.949997,211.710007,208.754997,222.295507,True
2023-07-24,235.419998,236.130005,233.919998,235.000000,1771300.0,0.178723,0.480853,2702980.0,238.479996,238.479996,218.440002,214.631996,229.362003,True
